In [ ]:
```python
# Test Results Notebook

# Import necessary libraries
import unittest
import sys
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from datetime import datetime

# Import test modules
from tests.test_preprocessing import TestPreprocessing
from tests.test_models import TestModels
from tests.test_prediction import TestPrediction

class TestRunner:
    def __init__(self):
        self.test_results = {}
        self.total_tests = 0
        self.passed_tests = 0
        self.failed_tests = 0
        self.error_tests = 0

    def run_test_suite(self, test_class, suite_name):
        """Run a test suite and collect results"""
        suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
        result = unittest.TestResult()
        suite.run(result)

        # Collect results
        suite_results = {
            'total': suite.countTestCases(),
            'passed': suite.countTestCases() - len(result.failures) - len(result.errors),
            'failed': len(result.failures),
            'errors': len(result.errors),
            'failures': result.failures,
            'error_details': result.errors
        }

        self.test_results[suite_name] = suite_results
        self.total_tests += suite_results['total']
        self.passed_tests += suite_results['passed']
        self.failed_tests += suite_results['failed']
        self.error_tests += suite_results['errors']

        return suite_results

    def display_results(self):
        """Display test results in a formatted way"""
        print("=== Test Results Summary ===")
        print(f"Total Tests: {self.total_tests}")
        print(f"Passed: {self.passed_tests}")
        print(f"Failed: {self.failed_tests}")
        print(f"Errors: {self.error_tests}")
        print("\nDetailed Results:")

        for suite_name, results in self.test_results.items():
            print(f"\n{suite_name}:")
            print(f"  Total: {results['total']}")
            print(f"  Passed: {results['passed']}")
            print(f"  Failed: {results['failed']}")
            print(f"  Errors: {results['errors']}")

            if results['failures']:
                print("\n  Failed Tests:")
                for failure in results['failures']:
                    print(f"    - {failure[0]}: {failure[1]}")

            if results['error_details']:
                print("\n  Test Errors:")
                for error in results['error_details']:
                    print(f"    - {error[0]}: {error[1]}")

    def plot_results(self):
        """Plot test results"""
        plt.figure(figsize=(15, 5))

        # Plot overall results
        plt.subplot(1, 2, 1)
        labels = ['Passed', 'Failed', 'Errors']
        sizes = [self.passed_tests, self.failed_tests, self.error_tests]
        colors = ['#2ecc71', '#e74c3c', '#f1c40f']
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%')
        plt.title('Overall Test Results')

        # Plot results by suite
        plt.subplot(1, 2, 2)
        suites = list(self.test_results.keys())
        passed = [r['passed'] for r in self.test_results.values()]
        failed = [r['failed'] for r in self.test_results.values()]
        errors = [r['errors'] for r in self.test_results.values()]

        x = np.arange(len(suites))
        width = 0.25

        plt.bar(x - width, passed, width, label='Passed', color='#2ecc71')
        plt.bar(x, failed, width, label='Failed', color='#e74c3c')
        plt.bar(x + width, errors, width, label='Errors', color='#f1c40f')

        plt.xlabel('Test Suites')
        plt.ylabel('Number of Tests')
        plt.title('Results by Test Suite')
        plt.xticks(x, suites, rotation=45)
        plt.legend()

        plt.tight_layout()
        plt.show()

def analyze_test_performance():
    """Analyze and display test performance metrics"""
    import time

    performance_results = {}

    # Test preprocessing performance
    start_time = time.time()
    test_suite = unittest.TestLoader().loadTestsFromTestCase(TestPreprocessing)
    test_suite.run(unittest.TestResult())
    performance_results['Preprocessing'] = time.time() - start_time

    # Test model performance
    start_time = time.time()
    test_suite = unittest.TestLoader().loadTestsFromTestCase(TestModels)
    test_suite.run(unittest.TestResult())
    performance_results['Models'] = time.time() - start_time

    # Test prediction performance
    start_time = time.time()
    test_suite = unittest.TestLoader().loadTestsFromTestCase(TestPrediction)
    test_suite.run(unittest.TestResult())
    performance_results['Prediction'] = time.time() - start_time

    # Plot performance results
    plt.figure(figsize=(10, 5))
    plt.bar(performance_results.keys(), performance_results.values())
    plt.title('Test Suite Performance')
    plt.xlabel('Test Suite')
    plt.ylabel('Execution Time (seconds)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return performance_results

def generate_test_report(runner, performance_results):
    """Generate a comprehensive test report"""
    report = pd.DataFrame({
        'Test Suite': list(runner.test_results.keys()),
        'Total Tests': [r['total'] for r in runner.test_results.values()],
        'Passed': [r['passed'] for r in runner.test_results.values()],
        'Failed': [r['failed'] for r in runner.test_results.values()],
        'Errors': [r['errors'] for r in runner.test_results.values()],
        'Execution Time': [performance_results[suite] for suite in runner.test_results.keys()]
    })

    display(report)

    # Save report
    report.to_csv('test_report.csv', index=False)
    print("\nTest report saved to 'test_report.csv'")

# Run all tests
runner = TestRunner()

print("Running Preprocessing Tests...")
runner.run_test_suite(TestPreprocessing, 'Preprocessing Tests')

print("\nRunning Model Tests...")
runner.run_test_suite(TestModels, 'Model Tests')

print("\nRunning Prediction Tests...")
runner.run_test_suite(TestPrediction, 'Prediction Tests')

# Display results
runner.display_results()

# Plot results
runner.plot_results()

# Analyze performance
performance_results = analyze_test_performance()
print("\nTest Suite Performance (seconds):")
for suite, time in performance_results.items():
    print(f"{suite}: {time:.3f}s")

# Generate report
generate_test_report(runner, performance_results)
```